In [1]:
import glob
from torch.utils.data import DataLoader
from custom_dataset_concat import MyDataset
import cv2
from share import *
import torchvision

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"


dataset = MyDataset("kin_hed2")
dataloader = DataLoader(dataset, num_workers=1, batch_size=1, shuffle=True)


def get_key(name: str):
    name = name.split(".")[1]
    name = name.split("_")[-1]
    name = name[3:]
    return int(name)


# only returns paths
# no actual images
def get_sequence():
    for batch in dataloader:
        meta_batch = batch["meta"]["file_name"]
        meta = meta_batch[0]
        meta = meta.split("/")[1]
        meta = meta.split("_")[:-1]
        meta = "_".join(meta)
        print(meta)
        styles = glob.glob("./data/kin_hed2/jpg/" + meta + "*")
        styles.sort(key=get_key)
        structures = glob.glob("./data/kin_hed2/hint/" + meta + "*")
        structures.sort(key=get_key)
        print(styles)
        print(structures)
        return styles, structures


def get_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img = img.astype(np.float32) / 255.0
    return img

logging improved.


In [2]:
import einops
import numpy as np
from ldm.modules.encoders.modules import FrozenClipImageEmbedder
import torch.nn.functional as F
import config
import gradio as gr
import random
import torch
from PIL import Image
from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler

/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


In [3]:
p = "./train_log/kin_hed_concat3/lightning_logs/version_1/checkpoints/epoch=1-step=165530.ckpt"
# p = './train_log/kin_hed_cross_3/lightning_logs/version_0/checkpoints/epoch=4-step=412223.ckpt'

model = create_model("./models/cldm_v15_concat.yaml").cpu()
model.load_state_dict(load_state_dict(p, location="cuda"))
model = model.cuda()
ddim_sampler = DDIMSampler(model)
frozenClipImageEmbedder = FrozenClipImageEmbedder()
frozenClipImageEmbedder = frozenClipImageEmbedder.cuda()

seed = 42
if seed == -1:
    seed = random.randint(0, 65535)
seed_everything(seed)

if config.save_memory:
    model.low_vram_shift(is_diffusing=False)

/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/distributed/_sharded_tensor/__init__.py:8: DeprecationWarning: torch.distributed._sharded_tensor will be deprecated, use torch.distributed._shard.sharded_tensor instead
  warnings.warn(


ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_

Global seed set to 42


In [9]:
num_samples = 1
ddim_steps = 20
strength = 1
eta = 0
scale = 1

styles, structures = get_sequence()
styles = np.array([get_img(style) for style in styles])
styles = styles.copy().astype(np.float32) / 127.5 - 1.0

structures = np.array([get_img(structure) for structure in structures])
structures = structures.astype(np.float32) / 255.0

hints = [ np.concatenate([structure, style], axis=-1) for (structure, style) in zip(structures, styles[:-1]) ]
hints = np.array(hints)

B, H, W, C = hints.shape

control = torch.from_numpy(hints.copy()).float().cuda()
control = einops.rearrange(control, "b h w c -> b c h w")

cond = {
    "c_concat": [control],
    "c_crossattn": [
        model.get_learned_conditioning(
            [""] * B
        )
    ],
}
un_cond = {
    "c_concat": [control],
    "c_crossattn": [model.get_learned_conditioning([""] * B)],
}
shape = (4, H // 8, W // 8)

model.control_scales = [
    strength
] * 13  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
samples, intermediates = ddim_sampler.sample(
    ddim_steps,
    B,
    shape,
    cond,
    verbose=False,
    eta=eta,
    unconditional_guidance_scale=scale,
    unconditional_conditioning=un_cond,
)



k0Zrqpv7V74_000073_000083
['./data/kin_hed2/jpg/k0Zrqpv7V74_000073_000083_idx0.png', './data/kin_hed2/jpg/k0Zrqpv7V74_000073_000083_idx500.png', './data/kin_hed2/jpg/k0Zrqpv7V74_000073_000083_idx1000.png']
['./data/kin_hed2/hint/k0Zrqpv7V74_000073_000083_idx500.png', './data/kin_hed2/hint/k0Zrqpv7V74_000073_000083_idx1000.png']
Data shape for DDIM sampling is (2, 4, 64, 64), eta 0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|██████████| 20/20 [00:02<00:00,  8.33it/s]


In [10]:
print(samples.shape)
d_samples = model.decode_first_stage(samples)
x_samples = einops.rearrange(d_samples, "b c h w -> b h w c") * 127.5 + 127.5

grid = torchvision.utils.make_grid(d_samples, nrow=1)
grid = grid.transpose(0, 1).transpose(1, 2).squeeze(-1)
grid = grid * 127.5 + 127.5
grid = grid.cpu().numpy().clip(0, 255).astype(np.uint8)

x_samples = x_samples.cpu().numpy().clip(0, 255).astype(np.uint8)

print(grid.shape)
print(x_samples[0].shape)
Image.fromarray(x_samples[0]).save("test_img1.png")
Image.fromarray(grid).save("test_grid.png")

torch.Size([2, 4, 64, 64])
(1030, 516, 3)
(512, 512, 3)


In [7]:
styles.shape

(5, 512, 512, 3)

In [8]:
structures.shape

(4, 512, 512, 3)